In [1]:
# Imports
import pandas as pd
from datetime import datetime
pd.options.display.max_colwidth = 100000

In [2]:
# Read pickle file with topics and augmented text
df4 = pd.read_pickle("data/allData.pkl")

# Make MIN_ALT and MAX_ALT numeric
df4['MIN_ALT'] = df4['MIN_ALT'].replace('MSL', 0)
df4['MAX_ALT'] = df4['MAX_ALT'].replace('MSL', 0)
df4['MIN_ALT'] = pd.to_numeric(df4['MIN_ALT'], errors='coerce')
df4['MAX_ALT'] = pd.to_numeric(df4['MAX_ALT'], errors='coerce')

# Make POSSIBLE_START_DATE, POSSIBLE_END_DATE, ISSUE_DATE, and CANCELED_DATE datetime format
df4['POSSIBLE_START_DATE'] = pd.to_datetime(df4['POSSIBLE_START_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['POSSIBLE_END_DATE'] = pd.to_datetime(df4['POSSIBLE_END_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['ISSUE_DATE'] = pd.to_datetime(df4['ISSUE_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['CANCELED_DATE'] = pd.to_datetime(df4['CANCELED_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')

# Replace missing POSSIBLE_START_DATE with CANCELED_DATE and vise versa
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_END_DATE']
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'CANCELED_DATE']

# Get mode of all NOTAM durations
est_days = (df4['POSSIBLE_END_DATE'] - df4['POSSIBLE_START_DATE']).mode()

# Fix remaining missing values of CANCELED_DATE and POSSIBLE_END_DATE by adding the mode to the POSSIBLE_START_DATE
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')

# Drop NOTAMs without POSSIBLE_END_DATE and POSSIBLE_START_DATE
df4 = df4.dropna( how='all', subset=['POSSIBLE_END_DATE', 'POSSIBLE_START_DATE'])

In [3]:
# Check missing values
df4.isna().sum()

NOTAM_REC_ID                    0
FNS_ID                          0
FILENAME                        0
NOTAM_ID                       26
NOTAM_TYPE                 221168
RELATED_NOTAM_ID          1281589
SIMPLE_TEXT                821736
LOC_ID_ACCOUNTABLE_ORG     864049
NOTAM_NUMBER               867227
RELATED_NOTAM_NUMBER      1524160
TEXT                            0
Q_CODE                     639786
Q_CODE_INTERPRETATION      909240
A_CODE                     508084
B_CODE                     666838
C_CODE                     820329
D_CODE                    1480096
E_CODE                        445
F_CODE                     629754
G_CODE                     629952
CLASSIFICATION                 31
POSSIBLE_NOTAM_ID              26
MIN_ALT                    461512
MAX_ALT                    514142
MIN_ALT_REF_TYPE           597577
MAX_ALT_REF_TYPE           665715
POSSIBLE_START_DATE             0
POSSIBLE_END_DATE               0
ISSUE_DATE                     26
CANCELED_DATE 

In [4]:
# One off testing and tuning

#time = '10/05/2017  08:00:00 AM' #349
#time = '2/20/2016  7:40:00 PM' #230
#time = '4/22/2020  7:30:30 PM' #514
time = '4/2/2018  8:30:38 PM' #391
#time = '11/16/2017  2:52:00 AM' #364

date_time_obj = datetime.strptime(time, '%m/%d/%Y %H:%M:%S %p')

In [5]:
# Date filter
filterA = df4[(df4['POSSIBLE_START_DATE'] <= time) & (df4['POSSIBLE_END_DATE'] >= time)]

In [6]:
# Altitude filter
filterB = filterA[filterA['MAX_ALT'] >= 50000 | filterA['MAX_ALT'].isna()]

In [7]:
# Keyword filter
filterC = filterB[(filterB['TEXT'].str.contains(r'(?:\s|^)rocket(?:\s|$)') == True) | 
                      (filterB['TEXT'].str.contains(r'(?:\s|^)space(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)launch(?:\s|$)') == True) |
                      #(filterB['TEXT'].str.contains(r'(?:\s|^)satellite(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)missile(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)canaveral(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)kennedy(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)nasa(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)unlimited(?:\s|$)') == True)]

In [8]:
# Read FIR Codes
df = pd.read_csv('data/NA_FIR_Codes.csv')

# Read US Airport Codes
#df2 = pd.read_csv('data/AirportData_Clean_20210629_Geocoded.csv')

In [9]:
# FIR Code filter
filterD = filterC[filterC['LOCATION_CODE'].isin(df['FIR']) | filterC['LOCATION_CODE'].isna()]

# US Airport Code filter
#filterD = filterC[filterC['LOCATION_CODE'].isin(df['FIR']) | filterC['LOCATION_CODE'].isna() | filterC['LOCATION_CODE'].isin(df2['locid'])]

In [10]:
# Print out
filterD[['TOPIC','NOTAM_REC_ID', 'TEXT']]

,TOPIC,NOTAM_REC_ID,TEXT
755589,7,755991,nj route zny zdc j cedar lake vcn vhf omnidirectional radio range distance measuring equipment nj to kennedy jfk vhf omnidirectional radio range distance measuring equipment ny maximum authorized (ifr) altitude estimate or estimation
755590,7,755992,nj route zdc zny j cedar lake vcn vhf omnidirectional radio range distance measuring equipment nj to kennedy jfk vhf omnidirectional radio range distance measuring equipment ny maximum authorized (ifr) altitude estimate or estimation
829018,3,829471,airspace unmanned rocket within an area defined altostratus nautical mile radius of swl surface unlimited daily
832833,3,833294,erop cross attention airline dispatchers some operations appear to be using the notice to airmen beginning time to start filing routes around the launch op launch hazard area times inch the notice to airmen are altitude specific and ditto not affect all operations this notice to airmen contains absolute (temperature) breakdown of the times for pre launch and launch closures the following airspace island inch use for absolute (temperature) missile launch splash down during these times kzma air route traffic control center oceanic area control zjx air route traffic control center and kzny air route traffic control center oceanic area control will not approve instrument flight rules flight within nautical mile of the affected oceanic stationary airspace reservations west of west or western longitude nautical mile between west or western longitude and west or western longitude and nautical mile east of west or western longitude or within affected warning area airspace effective west or western longitude absolute (temperature) west of west surface foot mean sea level effective cape air traffic control assigned airspace surface flight level effective restricted area (followed by identification) foot mean sea level unlimited effective restricted area (followed by identification) surface unlimited effective launch hazard area absolute (temperature) within an area defined altostratus north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to stopend (related to rvr) part of north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to point of origin surface unlimited launch hazard area blue within an area defined altostratus north or northern latitude west or western longitude north or northern latitude west or western longitude north or northern latitude west or western longitude to north or northern latitude west or western longitude to point of origin surface unlimited these areas encompass portions of restricted area (followed by identification) restricted area (followed by identification) and west or western longitude absolute (temperature) inch the interest of safety all nonparticipating pilots are strongly advised to avoid the above area except altostratus noted below for flight plan filing purposes the following restrictions will be inch effect within kzma and kzny firs all traffic regardless of altitude must be estimated to clear oceanic altitude reservation at or before kzny request aircraft certified for required navigation performance file routes to avoid hazard area blue blowing spray (weather reports only) nautical mile and non certify required navigation performance aircraft avoid area blue blowing spray (weather reports only) nautical mile during the hazard period jet terminal area jet terminal area the following routes will be impacted atlantic route atlantic route and mountain standard time (time groups only) the following routes will be impacted non required navigation performance aircraft only locator locator locator and mountain standard

In [11]:
# Topic filter
filterE = filterD[filterD['TOPIC'] == 3]

In [12]:
# Print out
filterE[['TOPIC','NOTAM_REC_ID', 'TEXT']]

,TOPIC,NOTAM_REC_ID,TEXT
829018,3,829471,airspace unmanned rocket within an area defined altostratus nautical mile radius of swl surface unlimited daily
832833,3,833294,erop cross attention airline dispatchers some operations appear to be using the notice to airmen beginning time to start filing routes around the launch op launch hazard area times inch the notice to airmen are altitude specific and ditto not affect all operations this notice to airmen contains absolute (temperature) breakdown of the times for pre launch and launch closures the following airspace island inch use for absolute (temperature) missile launch splash down during these times kzma air route traffic control center oceanic area control zjx air route traffic control center and kzny air route traffic control center oceanic area control will not approve instrument flight rules flight within nautical mile of the affected oceanic stationary airspace reservations west of west or western longitude nautical mile between west or western longitude and west or western longitude and nautical mile east of west or western longitude or within affected warning area airspace effective west or western longitude absolute (temperature) west of west surface foot mean sea level effective cape air traffic control assigned airspace surface flight level effective restricted area (followed by identification) foot mean sea level unlimited effective restricted area (followed by identification) surface unlimited effective launch hazard area absolute (temperature) within an area defined altostratus north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to stopend (related to rvr) part of north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to north or northern latitude west or western longitude to point of origin surface unlimited launch hazard area blue within an area defined altostratus north or northern latitude west or western longitude north or northern latitude west or western longitude north or northern latitude west or western longitude to north or northern latitude west or western longitude to point of origin surface unlimited these areas encompass portions of restricted area (followed by identification) restricted area (followed by identification) and west or western longitude absolute (temperature) inch the interest of safety all nonparticipating pilots are strongly advised to avoid the above area except altostratus noted below for flight plan filing purposes the following restrictions will be inch effect within kzma and kzny firs all traffic regardless of altitude must be estimated to clear oceanic altitude reservation at or before kzny request aircraft certified for required navigation performance file routes to avoid hazard area blue blowing spray (weather reports only) nautical mile and non certify required navigation performance aircraft avoid area blue blowing spray (weather reports only) nautical mile during the hazard period jet terminal area jet terminal area the following routes will be impacted atlantic route atlantic route and mountain standard time (time groups only) the following routes will be impacted non required navigation performance aircraft only locator locator locator and mountain standard time (time groups only) hibac transmit or transmitter shifty standard terminal arrival closed for flts filed over omn on the following stars fisel gissh frway caysl expect absolute (temperature) reroute by way of omn j trv and trv transition reroutes for atlantic route stopend (related to rvr) part of routes will be contained inch for an approach to air traffic control command center advisory
832834,3,833295,erop cross attention airline dispatchers some operations appear to be using the notice 

In [13]:
# Read in launch data
df2 = pd.read_csv('data/launches_20201027.csv', parse_dates=['LAUNCH_DATE'])

In [14]:
# Processing function
def getNotams(time):
    # Date filter
    filterA = df4[(df4['POSSIBLE_START_DATE'] <= time) & (df4['POSSIBLE_END_DATE'] >= time)]
    
    # Altitude filter
    filterB = filterA[filterA['MAX_ALT'] >= 50000 | filterA['MAX_ALT'].isna()]
    
    # FIR Code filter
    filterC = filterB[filterB['LOCATION_CODE'].isin(df['FIR']) | filterB['LOCATION_CODE'].isna()]
    
    # Keyword filter
    filterD = filterC[(filterC['TEXT'].str.contains(r'(?:\s|^)rocket(?:\s|$)') == True) | 
                      (filterC['TEXT'].str.contains(r'(?:\s|^)space(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)launch(?:\s|$)') == True) |
                      #(filterC['TEXT'].str.contains(r'(?:\s|^)satellite(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)missile(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)canaveral(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)kennedy(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)nasa(?:\s|$)') == True) |
                      (filterC['TEXT'].str.contains(r'(?:\s|^)unlimited(?:\s|$)') == True) ]
    
    # Topic filter
    filterE = filterD[filterD['TOPIC'] == 3]
    
    if len(filterE['NOTAM_REC_ID'] > 0):
        return filterE['NOTAM_REC_ID'].tolist()
    else:
        return

In [15]:
# Make predictions for launches
df2['DISCOVERED'] = df2['LAUNCH_DATE'].apply(getNotams)

In [16]:
# Final list of found launches
df2[~df2['DISCOVERED'].isnull().values]

,LAUNCHES_REC_ID,LAUNCH_DATE,VEHICLE_NAME,PAD_NAME,PAYLOAD,AGENCY,PURPOSE,ORBIT_TYPE,OUTCOME,SPACE_LAUNCH_REPORT_FLAG,GUNTER_FLAG,SPACE_FLIGHT_NOW_FLAG,NOONAN_FLAG,AST_FLAG,WIKIPEDIA_FLAG,JSR_FLAG,SPACEPORT_REC_ID,DISCOVERED
247,248,2016-05-26 21:40:00,Falcon 9 v1.2,SLC-40,Thiacom 8,SpaceX,Communications,Geosynchronous,Scrubbed,False,False,False,True,False,False,False,2.0,"[1006, 1076, 1121, 1140, 1155]"
248,249,2016-05-27 21:39:00,Falcon 9 v1.2,SLC-40,Thiacom 8,SpaceX,Communications,Geosynchronous,Operational,True,True,True,True,True,True,True,2.0,"[4527, 4679]"
253,254,2016-06-15 14:29:00,Falcon 9 v1.2,SLC-40,Eutelsat 117 West B,SpaceX,Communications,Geosynchronous,Operational,True,True,True,True,True,True,True,2.0,"[24942, 24943, 29952, 29954]"
256,257,2016-06-24 14:30:00,Atlas 5-551,SLC-41,MUOS 5,US Navy,Communications,Geosynchronous,Operational,True,True,True,True,False,True,True,2.0,"[39421, 39423, 43380, 48743, 48744]"
257,258,2016-07-18 04:45:29,Falcon 9 v1.2,SLC-40,Dragon/CRS-9,SpaceX,ISS Delivery,Low Earth,Operational,True,True,True,True,True,True,True,2.0,"[52822, 52823, 52824, 53429, 53430, 53432]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,512,2020-03-26 20:18:00,Atlas 5-551,SLC-41,AEHF 6,US Military,Communications,Supersynchronous,NaN,True,True,True,False,False,True,False,2.0,"[1612521, 1612522]"
513,514,2020-04-22 19:30:30,Falcon 9 v1.2,LC-39A,Starlink 1 F6,SpaceX,Communications,Low Earth,Operational,True,True,True,False,True,True,False,8.0,"[1629217, 1629218]"
514,515,2020-05-17 13:14:00,Atlas 5-501,SLC-41,USSF 7 (X37B OTV6),US Military,Testing,Low Earth,Operational,True,True,True,False,False,True,False,2.0,"[1634871, 1634872, 1634873, 1634874, 1634887, 1634888, 1634889, 1634899, 1634900]"
515,516,2020-05-25 18:50:00,LauncherOne,NaN,Starshine 4/Flight 1,Virgin Orbit,Testing,Low Earth,Failure,False,True,True,False,True,True,False,11.0,[1641222]


In [23]:
# Testing
df4[df4['NOTAM_REC_ID'] == 1612522]['TEXT']

1611809    cape canaveral flight level temporary flight restriction pursuant to code of federal regulations section flight limitation inch the proximity of space flight operations operations blowing spray (weather reports only) for an approach to certificate pilots or intensity unknown (weather reports only) south or southern latitude registration aircraft are prohibited within an area defined altostratus north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn then clockwise by way of absolute (temperature) nautical mile area chart centered at north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn to north or northern latitude west or western longitude omn to north or northern latitude we

In [18]:
# Testing
df2[df2['LAUNCHES_REC_ID'] == 391]

,LAUNCHES_REC_ID,LAUNCH_DATE,VEHICLE_NAME,PAD_NAME,PAYLOAD,AGENCY,PURPOSE,ORBIT_TYPE,OUTCOME,SPACE_LAUNCH_REPORT_FLAG,GUNTER_FLAG,SPACE_FLIGHT_NOW_FLAG,NOONAN_FLAG,AST_FLAG,WIKIPEDIA_FLAG,JSR_FLAG,SPACEPORT_REC_ID,DISCOVERED
390,391,2018-04-02 20:30:38,Falcon 9 v1.2,SLC-40,Dragon CRS-14,SpaceX,ISS Delivery,Low Earth,Successful,True,True,True,False,True,True,True,2.0,"[829471, 833294, 833295, 833296, 835580, 835581, 835582, 848684, 848685, 848686]"


In [19]:
# Testing
df3 = df2[['LAUNCHES_REC_ID', 'DISCOVERED']]

In [20]:
df3.loc[~df3['DISCOVERED'].isna()].head()

,LAUNCHES_REC_ID,DISCOVERED
247,248,"[1006, 1076, 1121, 1140, 1155]"
248,249,"[4527, 4679]"
253,254,"[24942, 24943, 29952, 29954]"
256,257,"[39421, 39423, 43380, 48743, 48744]"
257,258,"[52822, 52823, 52824, 53429, 53430, 53432]"


TODO
1. Find Related NOTAMs
2. Word Clouds
3. lda2vec??
4. Cluster TFIDF???
5. Classify model??

In [ ]:
import numpy as np
def findRelated(row):
    if row['DISCOVERED'] == None:
        return
    else:
        print(row['LAUNCHES_REC_ID'])
        print(row['DISCOVERED'], '\n')
        
        newNotams = []
        for item in row['DISCOVERED']:
            tmp = df4.loc[df4['NOTAM_REC_ID'] == item, 'RELATED_NOTAM_ID'].values
            
            print(np.isnan(tmp))
            
        print(newNotams)

In [ ]:
df3['RELATED'] = df3.apply (lambda row: findRelated(row), axis=1)